In [11]:
# import relevant packages
import numpy as np
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import numpy as np
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow_addons as tfa

In [12]:
dataset = load_dataset("paws", "labeled_final")

In [13]:
train_data = dataset['train']
valid_data = dataset['validation']
test_data = dataset['test']

In [14]:
def load_glove_embeddings(path):
    with open(path, 'r', encoding='utf-8') as f:
        embeddings = {}
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

In [15]:
path = 'glove.6B/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(path=path)

In [16]:
def get_cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [17]:
def get_sentence_embedding(sentence, embeddings_dict):
    embeddings = [embeddings_dict[word.lower()] for word in sentence.split() if word.lower() in embeddings_dict]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(next(iter(embeddings_dict.values())).shape)

In [18]:
def get_embeddings_and_labels(data, glove_embeddings):
    embeddings = []
    labels = []
    for example in data:
        sentence1_embedding = get_sentence_embedding(example['sentence1'], glove_embeddings)
        sentence2_embedding = get_sentence_embedding(example['sentence2'], glove_embeddings)
        embedding = np.concatenate((sentence1_embedding, sentence2_embedding))
        embeddings.append(embedding)
        labels.append(example['label'])
    return np.array(embeddings), np.array(labels)

In [19]:
word_freq = defaultdict(int)
for sentences in (train_data['sentence1'] + train_data['sentence2'] + \
                  test_data['sentence1'] + test_data['sentence2'] + \
                    valid_data['sentence1'] + valid_data['sentence2']):
    for word in sentences.split():
        word_freq[word.lower()] += 1
total_words = sum(word_freq.values())
word_prob = {word: freq / total_words for word, freq in word_freq.items()}

In [20]:
def get_sentence_embedding_sif(sentence, embeddings_dict, word_prob, a=1e-3):
    embeddings = []
    weights = []
    for word in sentence.split():
        if word.lower() in embeddings_dict and word.lower() in word_prob:
            embeddings.append(embeddings_dict[word.lower()])
            weight = a / (a + word_prob[word.lower()])
            weights.append(weight)
    if embeddings:
        weighted_embeddings = np.average(embeddings, axis=0, weights=weights)
        return weighted_embeddings
    else:
        return np.zeros(next(iter(embeddings_dict.values())).shape)

In [29]:
def get_embeddings_and_labels_sif(data, glove_embeddings):
    embeddings = []
    labels = []
    for example in data:
        sentence1_embedding = get_sentence_embedding_sif(example['sentence1'], glove_embeddings, word_prob=word_prob)
        sentence2_embedding = get_sentence_embedding_sif(example['sentence2'], glove_embeddings, word_prob=word_prob)
        embedding = np.concatenate((sentence1_embedding, sentence2_embedding))
        embeddings.append(embedding)
        labels.append(example['label'])
    return np.array(embeddings), np.array(labels)

In [21]:
all_sentences = train_data['sentence1'] + train_data['sentence2'] + \
                valid_data['sentence1'] + valid_data['sentence2'] + \
                test_data['sentence1'] + test_data['sentence2']
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_sentences)
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = dict(zip(feature_names, vectorizer.idf_))

In [22]:
def get_sentence_embedding_tfidf(sentence, embeddings_dict, tfidf_scores):
    embeddings = []
    weights = []
    for word in sentence.split():
        if word.lower() in embeddings_dict and word.lower() in tfidf_scores:
            embeddings.append(embeddings_dict[word.lower()])
            weight = tfidf_scores[word.lower()]
            weights.append(weight)
    if embeddings:
        if not weights:
            weights = [1] * len(embeddings)
        weighted_embeddings = np.average(embeddings, axis=0, weights=weights)
        return weighted_embeddings
    else:
        return np.zeros(next(iter(embeddings_dict.values())).shape)

In [23]:
def get_embeddings_and_labels_tfidf(data, embeddings_dict, a=1e-3):
    embeddings = []
    labels = []

    for sentence1, sentence2, label in zip(data['sentence1'], data['sentence2'], data['label']):
        sentence1_embedding = get_sentence_embedding_tfidf(sentence1, embeddings_dict, tfidf_scores)
        sentence2_embedding = get_sentence_embedding_tfidf(sentence2, embeddings_dict, tfidf_scores)

        embedding = np.concatenate((sentence1_embedding, sentence2_embedding))
        embeddings.append(embedding)
        labels.append(label)

    return np.array(embeddings), np.array(labels)

In [24]:
def get_eval_metrics(actual, predictions):
    accuracy = accuracy_score(actual, predictions)
    precision = precision_score(actual, predictions)
    recall = recall_score(actual, predictions)
    f1 = f1_score(actual, predictions)

    return accuracy, precision, recall, f1

Using a simple cosine similarity function and figuring out the threshold by optimising for the best threshold

In [44]:
len(train_data['label'])

49401

In [45]:
count = 0

for label in train_data['label']:
    if label == 0:
        count = count + 1

In [46]:
count

27572

In [41]:
best_threshold = None
best_metric_f1 = -1
best_metric_accuracy = -1
best_metric_recall = -1
best_metric_precision = -1

thresholds = [i * 0.05 for i in range(1, 39)]
for threshold in thresholds:
    predictions = []
    for p1, p2 in zip(train_data['sentence1'], train_data['sentence2']):
        emb1 = get_sentence_embedding_tfidf(p1, glove_embeddings, word_prob)
        emb2 = get_sentence_embedding_tfidf(p2, glove_embeddings, word_prob)
        similarity = get_cosine_similarity(emb1, emb2)
        pred = 1 if similarity > threshold else 0
        predictions.append(pred)
    accuracy, precision, recall, f1 = get_eval_metrics(train_data['label'], predictions)
    if accuracy > best_metric_accuracy:
        best_metric_f1 = f1
        best_metric_accuracy = accuracy
        best_metric_precision = precision
        best_metric_recall = recall
        best_threshold = threshold
print(f"The best threshold is {best_threshold}")

print(f"Training Accuracy: {best_metric_accuracy:.4f}")
print(f"Training Precision: {best_metric_precision:.4f}")
print(f"Training Recall: {best_metric_recall:.4f}")
print(f"Training F1-score: {best_metric_f1:.4f}")

/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

The best threshold is 1.0
Training Accuracy: 0.5581
Training Precision: 0.0000
Training Recall: 0.0000
Training F1-score: 0.0000


/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The best threshold with training set is coming out to be 0.05. Lets use this to see what sort of a score we would get with the validation set.

In [42]:
predictions = []
threshold = 0.65

for p1, p2 in zip(test_data['sentence1'], test_data['sentence2']):
    emb1 = get_sentence_embedding_tfidf(p1, glove_embeddings, word_prob)
    emb2 = get_sentence_embedding_tfidf(p2, glove_embeddings, word_prob)
    similarity = get_cosine_similarity(emb1, emb2)
    pred = 1 if similarity > threshold else 0
    predictions.append(pred)

actual = test_data['label']
accuracy, precision, recall, f1 = get_eval_metrics(actual, predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Test Accuracy: 0.4420
Test Precision: 0.4420
Test Recall: 1.0000
Test F1-score: 0.6130


In [28]:
train_embeddings, train_labels = get_embeddings_and_labels(train_data, glove_embeddings)
model = LogisticRegression(
    C=10,
    penalty='l1',
    solver='liblinear',
    max_iter=500,
    tol=1e-5,
    class_weight='balanced',
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5600
Training Precision: 0.5020
Training Recall: 0.5307
Training F1-score: 0.5159
Validation Accuracy: 0.5292
Validation Precision: 0.4700
Validation Recall: 0.5030
Validation F1-score: 0.4859
Test Accuracy: 0.5235
Test Precision: 0.4628
Test Recall: 0.4861
Test F1-score: 0.4742


In [30]:
train_embeddings, train_labels = get_embeddings_and_labels_sif(train_data, glove_embeddings)
model = LogisticRegression(
    C=10,
    penalty='l1',
    solver='liblinear',
    max_iter=500,
    tol=1e-5,
    class_weight='balanced',
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels_sif(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels_sif(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5546
Training Precision: 0.4963
Training Recall: 0.5355
Training F1-score: 0.5152
Validation Accuracy: 0.5244
Validation Precision: 0.4655
Validation Recall: 0.5066
Validation F1-score: 0.4852
Test Accuracy: 0.5272
Test Precision: 0.4675
Test Recall: 0.5008
Test F1-score: 0.4836


In [31]:
train_embeddings, train_labels = get_embeddings_and_labels_tfidf(train_data, glove_embeddings)
model = LogisticRegression(
    C=10,
    penalty='l1',
    solver='liblinear',
    max_iter=500,
    tol=1e-5,
    class_weight='balanced',
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels_tfidf(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels_tfidf(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5532
Training Precision: 0.4948
Training Recall: 0.5320
Training F1-score: 0.5127
Validation Accuracy: 0.5280
Validation Precision: 0.4693
Validation Recall: 0.5126
Validation F1-score: 0.4900
Test Accuracy: 0.5261
Test Precision: 0.4663
Test Recall: 0.4986
Test F1-score: 0.4819


In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

train_embeddings, train_labels = get_embeddings_and_labels_tfidf(train_data, glove_embeddings)

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1_score': make_scorer(f1_score, average='macro')}

model = LogisticRegression(
    solver='saga',
    max_iter=1000,
    tol=1e-5,
    class_weight='balanced',
    random_state=42
)

grid_search = GridSearchCV(model, param_grid, scoring=scoring, refit='f1_score', cv=5, verbose=1)
grid_search.fit(train_embeddings, train_labels)
print("Best parameters found: ", grid_search.best_params_)
print("Best F1-score achieved: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
train_predictions = best_model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels_tfidf(valid_data, glove_embeddings)
valid_predictions = best_model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels_tfidf(test_data, glove_embeddings)
test_predictions = best_model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarni

Best parameters found:  {'C': 0.1, 'penalty': 'l1'}
Best F1-score achieved:  0.5264764044620778
Training Accuracy: 0.5430
Training Precision: 0.4840
Training Recall: 0.5175
Training F1-score: 0.5002
Validation Accuracy: 0.5259
Validation Precision: 0.4648
Validation Recall: 0.4733
Validation F1-score: 0.4690
Test Accuracy: 0.5230
Test Precision: 0.4621
Test Recall: 0.4833
Test F1-score: 0.4725


In [32]:
train_embeddings, train_labels = get_embeddings_and_labels(train_data, glove_embeddings)
model = SVC(
    C=1.0,
    kernel='sigmoid',
    degree=3,
    gamma='auto',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=1e-3,
    cache_size=200,
    class_weight='balanced',
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5581
Training Precision: 0.4999
Training Recall: 0.3944
Training F1-score: 0.4410
Validation Accuracy: 0.5489
Validation Precision: 0.4867
Validation Recall: 0.3606
Validation F1-score: 0.4142
Test Accuracy: 0.5465
Test Precision: 0.4837
Test Recall: 0.3849
Test F1-score: 0.4287


In [33]:
train_embeddings, train_labels = get_embeddings_and_labels_sif(train_data, glove_embeddings)
model = SVC(
    C=1.0,
    kernel='sigmoid',
    degree=3,
    gamma='auto',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=1e-3,
    cache_size=200,
    class_weight='balanced',
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels_sif(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels_sif(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5477
Training Precision: 0.4869
Training Recall: 0.4399
Training F1-score: 0.4622
Validation Accuracy: 0.5389
Validation Precision: 0.4750
Validation Recall: 0.4032
Validation F1-score: 0.4362
Test Accuracy: 0.5369
Test Precision: 0.4740
Test Recall: 0.4361
Test F1-score: 0.4543


In [34]:
train_embeddings, train_labels = get_embeddings_and_labels_tfidf(train_data, glove_embeddings)

model = SVC(
    C=1.0,
    kernel='sigmoid',
    degree=3,
    gamma='auto',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=1e-3,
    cache_size=200,
    class_weight='balanced',
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=42
)
model.fit(train_embeddings, train_labels)

train_predictions = model.predict(train_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(train_labels, train_predictions)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training Precision: {precision:.4f}")
print(f"Training Recall: {recall:.4f}")
print(f"Training F1-score: {f1:.4f}")

valid_embeddings, valid_labels = get_embeddings_and_labels_tfidf(valid_data, glove_embeddings)
valid_predictions = model.predict(valid_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(valid_labels, valid_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

test_embeddings, test_labels = get_embeddings_and_labels_tfidf(test_data, glove_embeddings)
test_predictions = model.predict(test_embeddings)
accuracy, precision, recall, f1 = get_eval_metrics(test_labels, test_predictions)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-score: {f1:.4f}")

Training Accuracy: 0.5461
Training Precision: 0.4854
Training Recall: 0.4547
Training F1-score: 0.4696
Validation Accuracy: 0.5383
Validation Precision: 0.4756
Validation Recall: 0.4267
Validation F1-score: 0.4498
Test Accuracy: 0.5344
Test Precision: 0.4718
Test Recall: 0.4465
Test F1-score: 0.4588


In [40]:
train_embeddings, train_labels = get_embeddings_and_labels_sif(train_data, glove_embeddings)
valid_embeddings, valid_labels = get_embeddings_and_labels_sif(valid_data, glove_embeddings)



METRICS = [
    tf.keras.metrics.AUC(name='roc-auc'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.F1Score(name='f1_score', threshold=0.5, num_classes=1)  # Add F1 score
]

model = Sequential([
    Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

history = model.fit(train_embeddings, train_labels, epochs=100,
                    validation_data=(valid_embeddings, valid_labels))

model.evaluate(valid_embeddings, valid_labels)

Epoch 1/100
1544/1544 [==============================] - 2s 887us/step - loss: 0.6849 - roc-auc: 0.5299 - accuracy: 0.5613 - precision: 0.5143 - recall: 0.1298 - f1_score: 0.2073 - val_loss: 0.6868 - val_roc-auc: 0.5329 - val_accuracy: 0.5541 - val_precision: 0.4892 - val_recall: 0.1800 - val_f1_score: 0.2632
Epoch 2/100
1544/1544 [==============================] - 1s 791us/step - loss: 0.6779 - roc-auc: 0.5509 - accuracy: 0.5777 - precision: 0.5855 - recall: 0.1515 - f1_score: 0.2407 - val_loss: 0.6876 - val_roc-auc: 0.5267 - val_accuracy: 0.5534 - val_precision: 0.4794 - val_recall: 0.1119 - val_f1_score: 0.1814
Epoch 3/100
1544/1544 [==============================] - 1s 846us/step - loss: 0.6728 - roc-auc: 0.5617 - accuracy: 0.5850 - precision: 0.6022 - recall: 0.1791 - f1_score: 0.2761 - val_loss: 0.6840 - val_roc-auc: 0.5481 - val_accuracy: 0.5598 - val_precision: 0.5072 - val_recall: 0.1681 - val_f1_score: 0.2525
Epoch 4/100
1544/1544 [==============================] - 1s 789us/s

[0.8604294061660767,
 0.5149820446968079,
 0.5394999980926514,
 0.47146791219711304,
 0.33851370215415955,
 array([0.39407897], dtype=float32)]

In [36]:
train_embeddings, train_labels = get_embeddings_and_labels_tfidf(train_data, glove_embeddings)
valid_embeddings, valid_labels = get_embeddings_and_labels_tfidf(valid_data, glove_embeddings)


METRICS = [
    tf.keras.metrics.AUC(name='roc-auc'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.F1Score(name='f1_score', threshold=0.5, num_classes=1)
]

model = Sequential([
    Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

history = model.fit(train_embeddings, train_labels, epochs=10,
                    validation_data=(valid_embeddings, valid_labels))

model.evaluate(valid_embeddings, valid_labels)

Epoch 1/10
1544/1544 [==============================] - 2s 830us/step - loss: 0.6848 - roc-auc: 0.5281 - accuracy: 0.5621 - precision: 0.5175 - recall: 0.1330 - f1_score: 0.2116 - val_loss: 0.6867 - val_roc-auc: 0.5340 - val_accuracy: 0.5533 - val_precision: 0.4868 - val_recall: 0.1825 - val_f1_score: 0.2655
Epoch 2/10
1544/1544 [==============================] - 1s 732us/step - loss: 0.6768 - roc-auc: 0.5549 - accuracy: 0.5806 - precision: 0.5830 - recall: 0.1785 - f1_score: 0.2733 - val_loss: 0.6850 - val_roc-auc: 0.5414 - val_accuracy: 0.5636 - val_precision: 0.5208 - val_recall: 0.1698 - val_f1_score: 0.2561
Epoch 3/10
1544/1544 [==============================] - 1s 740us/step - loss: 0.6713 - roc-auc: 0.5642 - accuracy: 0.5882 - precision: 0.6096 - recall: 0.1894 - f1_score: 0.2890 - val_loss: 0.6860 - val_roc-auc: 0.5326 - val_accuracy: 0.5648 - val_precision: 0.5376 - val_recall: 0.1150 - val_f1_score: 0.1895
Epoch 4/10
1544/1544 [==============================] - 1s 733us/step 

[0.6978882551193237,
 0.5460283160209656,
 0.562250018119812,
 0.5119896531105042,
 0.2232269048690796,
 array([0.31090122], dtype=float32)]

In [37]:
train_embeddings, train_labels = get_embeddings_and_labels(train_data, glove_embeddings)
valid_embeddings, valid_labels = get_embeddings_and_labels(valid_data, glove_embeddings)


METRICS = [
    tf.keras.metrics.AUC(name='roc-auc'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.F1Score(name='f1_score', threshold=0.5, num_classes=1)
]

model = Sequential([
    Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

history = model.fit(train_embeddings, train_labels, epochs=10,
                    validation_data=(valid_embeddings, valid_labels))

model.evaluate(valid_embeddings, valid_labels)

Epoch 1/10
1544/1544 [==============================] - 2s 821us/step - loss: 0.6841 - roc-auc: 0.5384 - accuracy: 0.5630 - precision: 0.5197 - recall: 0.1468 - f1_score: 0.2290 - val_loss: 0.6821 - val_roc-auc: 0.5504 - val_accuracy: 0.5650 - val_precision: 0.5774 - val_recall: 0.0622 - val_f1_score: 0.1122
Epoch 2/10
1544/1544 [==============================] - 1s 737us/step - loss: 0.6782 - roc-auc: 0.5558 - accuracy: 0.5756 - precision: 0.5685 - recall: 0.1636 - f1_score: 0.2541 - val_loss: 0.6812 - val_roc-auc: 0.5544 - val_accuracy: 0.5648 - val_precision: 0.5272 - val_recall: 0.1563 - val_f1_score: 0.2411
Epoch 3/10
1544/1544 [==============================] - 1s 792us/step - loss: 0.6745 - roc-auc: 0.5634 - accuracy: 0.5798 - precision: 0.5899 - recall: 0.1610 - f1_score: 0.2530 - val_loss: 0.6818 - val_roc-auc: 0.5559 - val_accuracy: 0.5675 - val_precision: 0.5755 - val_recall: 0.0851 - val_f1_score: 0.1482
Epoch 4/10
1544/1544 [==============================] - 1s 794us/step 

[0.6892088651657104,
 0.5548983216285706,
 0.5732499957084656,
 0.5812743902206421,
 0.12630686163902283,
 array([0.20752089], dtype=float32)]